In [1]:
#Import the libraries
import numpy as np
import pandas as pd

In [2]:
#Place dataset in DataFrame
_raw_data = """Outlook,Temperature,Humidity,Wind,Play
Sunny,Hot,High,Weak,No
Sunny,Hot,High,Strong,No
Overcast,Hot,High,Weak,Yes
Rain,Mild,High,Weak,Yes
Rain,Cool,Normal,Weak,Yes
Overcast,Cool,Normal,Strong,Yes
Sunny,Cool,Normal,Strong,Yes
Sunny,Mild,High,Weak,No
Rain,Cool,Normal,Weak,Yes
Sunny,Mild,Normal,Weak,Yes
Overcast,Mild,Normal,Strong,Yes
Overcast,Hot,Normal,Strong,Yes
"""

with open('sport_data.csv', 'w') as f:
    f.write(_raw_data)
df = pd.read_csv('sport_data.csv')

In [3]:
#Display the sports dataset as a DataFrame
df

,Outlook,Temperature,Humidity,Wind,Play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Overcast,Cool,Normal,Strong,Yes
6,Sunny,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Rain,Cool,Normal,Weak,Yes
9,Sunny,Mild,Normal,Weak,Yes


In [38]:
#Compute the entropy
def compute_entropy(y):
    if len(y) < 2:
        return 0
    freq = np.array(y.value_counts(normalize=True))
    return -(freq * np.log2(freq + 1e-6)).sum()

#Compute the info gain
def compute_info_gain(samples, attr, target):
    values = samples[attr].value_counts(normalize=True)
    split_ent = 0
    for v, fr in values.iteritems():
        sub_ent = compute_entropy(
            samples[samples[attr]==v][target])
        split_ent += fr * sub_ent

    ent = compute_entropy(samples[target])
    return ent - split_ent

In [39]:
#Entropy of each attribute
attr_ent = {v:compute_entropy(df[v]) for v in df.keys()[:-1]}
attr_ent

{'Outlook': 1.5545808412594577,
 'Temperature': 1.5849581726425257,
 'Humidity': 0.9798658712640389,
 'Wind': 0.9798658712640389}

In [40]:
#Information gain of each attribute
attr_ig = {v:compute_info_gain(df, v, 'Play') for v in df.keys()[:-1]}
attr_ig

{'Outlook': 0.4067145351997543,
 'Temperature': 0.2075182687424375,
 'Humidity': 0.40671453519975437,
 'Wind': 0.006987753258770657}

In [44]:
#The main part of the algorithm, creating the tree node
class TreeNode:
    #Constructor of the tree node
    def __init__(self, samples, target):
        self.decision = None
        self.samples = samples
        self.target = target
        self.split_attribute = None
        self.children = None
        
    def pretty_print(self, prefix=''):
        if self.split_attribute is not None:
            for k, v in self.children.items():
                v.pretty_print(f'{prefix}: When {self.split_attribute} is {k}')
        else:
            print(f'{prefix}: {self.decision}')
    
    def make(self):
        target = self.target
        samples = self.samples
        #Empty population, therefore retain the decision of parent decision
        if len(samples) == 0:
            self.decision = self.parent.decision
            return
        #Checks if there is only 1 unique value
        elif len(samples[target].unique()) == 1:
            #Extract the item from the array
            self.decision = samples[target].unique()[0]
            return
        else: #Otherwise, split the population of the dataset samples
            ig_max = 0
            for a in samples.keys(): #Loop over the attributes in the dataset
                #Skip over the 'target' attribute as we want to predict this
                if a == target:
                    continue
                aig = compute_info_gain(samples, a, target)
                if aig > ig_max: #Get the attribute with the maximum info gain
                    ig_max = aig
                    #Split the dataset by the attribute with highest info gain
                    self.split_attribute = a
            self.children = {}
            #Loop over the unique values of the attribute
            for v in samples[self.split_attribute].unique():
                index = samples[self.split_attribute] == v
                #Recursive part of the function
                #Create a children node with unique value
                self.children[v] = TreeNode(samples[index], target)
                self.children[v].make()

#Template for building the decision tree
class TreeID3:
    #Constructor of the tree
    def __init__(self):
        self.root = None

    #Apply the decision tree to the training dataset
    #As well as to initialise the root node of the tree
    def fit(self, samples, target):
        self.root = TreeNode(samples, target)
        self.root.make()

In [45]:
t = TreeID3()
t.fit(df, 'Play')
t.root.pretty_print()

: When Humidity is High: When Outlook is Sunny: No
: When Humidity is High: When Outlook is Overcast: Yes
: When Humidity is High: When Outlook is Rain: Yes
: When Humidity is Normal: Yes
